In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1653,tt0338325,20000000,16605763,Paparazzi,Cole Hauser|Robin Tunney|Dennis Farina|Daniel ...,Paul Abascal,One good shot deserves another.,A rising Hollywood actor decides to take perso...,84,Action|Drama|Thriller,20th Century Fox,9/3/2004,5.9,2004
661,tt0481536,12000000,43493123,Harold & Kumar Escape from Guantanamo Bay,John Cho|Kal Penn|Neil Patrick Harris|Rob Cord...,Jon Hurwitz|Hayden Schlossberg,This time they're running from the joint.,"Having satisfied their urge for White Castle, ...",107,Comedy|Adventure,New Line Cinema,4/25/2008,6.0,2008
1145,tt0301470,17000000,63102666,Jeepers Creepers 2,Ray Wise|Jonathan Breck|Garikayi Mutambirwa|Er...,Victor Salva,He can taste your fear.,After 23 horrifying days of gorging on human f...,104,Horror|Thriller,United Artists|American Zoetrope|Myriad Pictur...,8/8/2003,5.4,2003
1781,tt0841046,35000000,18317151,Walk Hard: The Dewey Cox Story,John C. Reilly|Jenna Fischer|Tim Meadows|Krist...,Jake Kasdan,Life made him tough. Love made him strong. Mus...,Singer Dewey Cox overcomes adversity to become...,96,Comedy|Music,Columbia Pictures Corporation|Apatow Productions,12/21/2007,6.5,2007
197,tt2369135,66000000,203277636,Need for Speed,Aaron Paul|Dominic Cooper|Imogen Poots|Rami Ma...,Scott Waugh,For honor. For love. For redemption.,The film revolves around a local street-racer ...,130,Action|Crime|Drama|Thriller,DreamWorks SKG|Reliance Entertainment|Bandito ...,3/13/2014,6.2,2014


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


# Предобработка

In [5]:
answers = {} # создадим словарь для ответов

# profit = revenue - budget was added, as not given in the original dataset
data['profit'] = data.revenue - data.budget 

#Possible way to split the date pf release
data[['month','day','year']] = data.release_date.str.split('/', expand=True) 

# Another way with type datetime
data['timefunc']= pd.to_datetime(data.release_date)

# required for 24th question: length of title:
data['title_length'] = data['original_title'].map(lambda x: len(x))

#created additional clolumn: number of words in tagline
data['num_words'] = data['overview'].map(lambda x: len(x.split(' '))) 

#function to split to the list with separatior '|'
def func(column):
    return str(column).split('|')

#function to combinate artists pairs:
import itertools as it
def func_comb(column):
    return list(it.combinations(column,2))

data.head()

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,month,day,year,timefunc,title_length,num_words
0,tt0369610,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,...,6/9/2015,6.5,2015,1363528810,6,9,2015,2015-06-09,14,26
1,tt1392190,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,...,5/13/2015,7.1,2015,228436354,5,13,2015,2015-05-13,18,110
2,tt2908446,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,...,3/18/2015,6.3,2015,185238201,3,18,2015,2015-03-18,9,22
3,tt2488496,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,...,12/15/2015,7.5,2015,1868178225,12,15,2015,2015-12-15,28,26
4,tt2820852,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,...,4/1/2015,7.3,2015,1316249360,4,1,2015,2015-04-01,9,14


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [6]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
# answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650) +'
# если ответили верно, можете добавить комментарий со значком "+"

In [7]:
# тут пишем ваш код для решения данного вопроса:
data.loc[data.budget == data.budget.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,month,day,year,timefunc,title_length,num_words
723,tt1298650,380000000,1021683000,Pirates of the Caribbean: On Stranger Tides,Johnny Depp|PenÃ©lope Cruz|Geoffrey Rush|Ian M...,Rob Marshall,Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,Adventure|Action|Fantasy,...,5/11/2011,6.3,2011,641683000,5,11,2011,2011-05-11,43,75


ВАРИАНТ 2

In [8]:
data.sort_values(by = ['budget'], ascending=False).head(1)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,month,day,year,timefunc,title_length,num_words
723,tt1298650,380000000,1021683000,Pirates of the Caribbean: On Stranger Tides,Johnny Depp|PenÃ©lope Cruz|Geoffrey Rush|Ian M...,Rob Marshall,Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,Adventure|Action|Fantasy,...,5/11/2011,6.3,2011,641683000,5,11,2011,2011-05-11,43,75


# 2. Какой из фильмов самый длительный (в минутах)?

In [9]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = '2. Gods and Generals (tt0279111) +'

In [10]:
data.loc[data.runtime == data.runtime.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,month,day,year,timefunc,title_length,num_words
1157,tt0279111,56000000,12923936,Gods and Generals,Stephen Lang|Jeff Daniels|Robert Duvall|Kevin ...,Ronald F. Maxwell,The nations heart was touched by...,The film centers mostly around the personal an...,214,Drama|History|War,...,2/21/2003,5.8,2003,-43076064,2,21,2003,2003-02-21,17,48


# 3. Какой из фильмов самый короткий (в минутах)?





In [11]:
data.loc[data.runtime == data.runtime.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,month,day,year,timefunc,title_length,num_words
768,tt1449283,30000000,14460000,Winnie the Pooh,Jim Cummings|Travis Oates|Jim Cummings|Bud Luc...,Stephen Anderson|Don Hall,Oh Pooh.,"During an ordinary day in Hundred Acre Wood, W...",63,Animation|Family,...,4/13/2011,6.8,2011,-15540000,4,13,2011,2011-04-13,15,52


In [12]:
answers['3'] = '3. Winnie the Pooh (tt1449283) +'

# 4. Какова средняя длительность фильмов?


In [13]:
round(data.runtime.mean())

110

In [14]:
data.describe().runtime['mean']

109.6585494970884

In [15]:
answers['4'] = '2. 110 +'

# 5. Каково медианное значение длительности фильмов? 

In [16]:
round(data.runtime.median())

107

In [17]:
data.describe().runtime['50%']

107.0

In [18]:
answers['5'] = '1. 107 +'
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650) +',
 '2': '2. Gods and Generals (tt0279111) +',
 '3': '3. Winnie the Pooh (tt1449283) +',
 '4': '2. 110 +',
 '5': '1. 107 +'}

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [19]:
# лучше код получения столбца profit вынести в Предобработку что в начале
data.loc[data.profit == data.profit.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,month,day,year,timefunc,title_length,num_words
239,tt0499549,237000000,2781505847,Avatar,Sam Worthington|Zoe Saldana|Sigourney Weaver|S...,James Cameron,Enter the World of Pandora.,"In the 22nd century, a paraplegic Marine is di...",162,Action|Adventure|Fantasy|Science Fiction,...,12/10/2009,7.1,2009,2544505847,12,10,2009,2009-12-10,6,28


In [20]:
data.sort_values(by='profit', ascending=False).head(1)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,month,day,year,timefunc,title_length,num_words
239,tt0499549,237000000,2781505847,Avatar,Sam Worthington|Zoe Saldana|Sigourney Weaver|S...,James Cameron,Enter the World of Pandora.,"In the 22nd century, a paraplegic Marine is di...",162,Action|Adventure|Fantasy|Science Fiction,...,12/10/2009,7.1,2009,2544505847,12,10,2009,2009-12-10,6,28


In [21]:
answers['6'] = '5. Avatar (tt0499549) +'

# 7. Какой фильм самый убыточный?

In [22]:
data.sort_values(by='profit').head(1)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,month,day,year,timefunc,title_length,num_words
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,...,7/3/2013,6.0,2013,-165710090,7,3,2013,2013-07-03,15,68


In [23]:
data.loc[data.profit == data.profit.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,month,day,year,timefunc,title_length,num_words
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,...,7/3/2013,6.0,2013,-165710090,7,3,2013,2013-07-03,15,68


In [24]:
answers['7'] = '5. The Lone Ranger (tt1210819) +'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [25]:
len(data[data.revenue > data.budget])

1478

In [26]:
data[data.revenue > data.budget].describe() #count - is answer, also check of the filtered data

,budget,revenue,runtime,vote_average,release_year,profit,title_length,num_words
count,1.478000e+03,1.478000e+03,1478.000000,1478.000000,1478.000000,1.478000e+03,1478.000000,1478.000000
mean,5.742285e+07,1.915854e+08,110.126522,6.240122,2008.039242,1.341626e+08,15.292287,53.547361
std,5.170014e+07,2.295625e+08,18.288838,0.738106,4.479981,1.941708e+08,8.817403,26.862752
min,5.000000e+06,8.203235e+06,72.000000,3.400000,2000.000000,2.000000e+06,1.000000,10.000000
25%,2.000000e+07,5.903202e+07,97.000000,5.800000,2004.000000,2.724810e+07,9.000000,34.000000
50%,4.000000e+07,1.130136e+08,107.000000,6.200000,2008.000000,6.709555e+07,13.000000,50.000000
75%,7.875000e+07,2.218417e+08,121.000000,6.700000,2012.000000,1.507748e+08,19.000000,67.000000
max,3.800000e+08,2.781506e+09,201.000000,8.100000,2015.000000,2.544506e+09,83.000000,169.000000


In [27]:
answers['8'] = '1. 1478 +'

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [28]:
data.loc[(data.release_year == 2008)].sort_values(by='profit', ascending=False).head()

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,month,day,year,timefunc,title_length,num_words
599,tt0468569,185000000,1001921825,The Dark Knight,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,Drama|Action|Crime|Thriller,...,7/16/2008,8.1,2008,816921825,7,16,2008,2008-07-16,15,67
603,tt0367882,185000000,786636033,Indiana Jones and the Kingdom of the Crystal S...,Harrison Ford|Cate Blanchett|Shia LaBeouf|Ray ...,Steven Spielberg,The adventure continues . . .,"Set during the Cold War, the Soviets â€“ led b...",122,Adventure|Action,...,5/21/2008,5.6,2008,601636033,5,21,2008,2008-05-21,50,74
607,tt0795421,52000000,609841637,Mamma Mia!,Meryl Streep|Pierce Brosnan|Amanda Seyfried|Co...,Phyllida Lloyd,Take a trip down the aisle you'll never forget,"Set on an idyllic Greek island, the plot serve...",108,Comedy|Romance,...,6/30/2008,6.2,2008,557841637,6,30,2008,2008-06-30,10,63
606,tt0441773,130000000,631744560,Kung Fu Panda,Jack Black|Dustin Hoffman|Angelina Jolie|Jacki...,Mark Osborne|John Stevenson,Prepare for awesomeness.,"When the Valley of Peace is threatened, lazy P...",90,Animation|Family,...,6/4/2008,6.8,2008,501744560,6,4,2008,2008-06-04,13,60
621,tt0448157,150000000,624029371,Hancock,Will Smith|Charlize Theron|Jason Bateman|Jae H...,Peter Berg,Bad Behavior. Bad Attitude. Real Hero.,Hancock is a down-and-out superhero who's forc...,92,Fantasy|Action,...,7/1/2008,6.0,2008,474029371,7,1,2008,2008-07-01,7,63


In [29]:
data2008 = data.loc[(data.release_year == 2008)] # new df for films released in 2008
data2008.loc[(data2008.profit == data2008.profit.max())] # - answer

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,month,day,year,timefunc,title_length,num_words
599,tt0468569,185000000,1001921825,The Dark Knight,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,Drama|Action|Crime|Thriller,...,7/16/2008,8.1,2008,816921825,7,16,2008,2008-07-16,15,67


In [30]:
answers['9'] = '4. The Dark Knight (tt0468569) +'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [31]:
data[(data.release_year >= 2012) & (data.release_year <= 2014)].sort_values(by='profit').head(1)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,month,day,year,timefunc,title_length,num_words
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,...,7/3/2013,6.0,2013,-165710090,7,3,2013,2013-07-03,15,68


In [32]:
answers['10'] = '5. The Lone Ranger (tt1210819) +'

# 11. Какого жанра фильмов больше всего?

In [33]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале
pd.Series(data['genres'].str.cat(sep='|').split('|')).value_counts()

Drama              782
Comedy             683
Thriller           596
Action             582
Adventure          415
Crime              315
Romance            308
Family             260
Science Fiction    248
Fantasy            222
Horror             176
Mystery            168
Animation          139
Music               64
History             62
War                 58
Western             19
Documentary          8
Foreign              2
dtype: int64

In [34]:
answers['11'] = '3. Drama +'

ВАРИАНТ 2

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [35]:
profit_df = data[data.profit>0].copy()
pd.Series(profit_df['genres'].str.cat(sep='|').split('|')).value_counts()

Drama              560
Comedy             551
Thriller           446
Action             444
Adventure          337
Romance            242
Crime              231
Family             226
Science Fiction    195
Fantasy            188
Horror             150
Animation          120
Mystery            119
Music               47
History             46
War                 41
Western             12
Documentary          7
dtype: int64

In [36]:
answers['12'] = '1. Drama +'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [37]:
data.pivot_table(values = 'revenue', index = 'director', aggfunc='sum').sort_values(by='revenue',ascending=False)
# Profit is not splited between the directorsof the single movie

,revenue
director,
Peter Jackson,6490593685
Christopher Nolan,4167548502
David Yates,4154295625
Michael Bay,3886938960
J.J. Abrams,3579169916
...,...
David MichÃ´d,2295423
Steven Shainberg,2281089
Paul Schrader,2062066


In [38]:
answers['13'] = '5. Peter Jackson +'

In [39]:
#Example from Slack
# for director in directors.index:
#     directors[director] = data['revenue'][data['director'].map(lambda x: True if director in x else False)].sum()
# directors.sort_values(ascending=False)
# Profit is not splitted between the directorsof the single movie

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [40]:
action_df = data[data.genres.str.contains('Action')].copy() # data sorted for Action genres
pd.Series(action_df['director'].str.cat(sep='|').split('|')).value_counts() #Directors separated and counted

Robert Rodriguez      9
Paul W.S. Anderson    7
Michael Bay           7
Antoine Fuqua         6
Ridley Scott          6
                     ..
Steven Quale          1
Daniel Espinosa       1
Patrick Tatopoulos    1
Sean McNamara         1
John Herzfeld         1
Length: 364, dtype: int64

In [41]:
answers['14'] = '3. Robert Rodriguez +'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [42]:
artist_df = data[data.release_year == 2012].copy() #DF for filns released in 2012
artist_df['cast'] = artist_df.cast.str.split('|')
artist_df.explode('cast').pivot_table(values='revenue', index='cast', aggfunc='sum').sort_values(by='revenue',ascending=False)


,revenue
cast,
Chris Hemsworth,2027450773
Denis Leary,1629460639
Anne Hathaway,1522851057
Robert Downey Jr.,1519557910
Mark Ruffalo,1519557910
...,...
Michael Nyqvist,3428048
Danny Huston,2106557
Josh Lucas,2106557


In [43]:
answers['15'] = '3. Chris Hemsworth +'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [44]:
budget_df = data.loc[data.budget > data.budget.mean()].copy() # New DF for movies with budget > mean budget. (for big data is not usefull)
budget_df['cast'] = budget_df.cast.str.split('|') # Split of the cast into the list
budget_df.explode('cast').value_counts('cast') # adding lines for each artist and counting

cast
Matt Damon           18
Adam Sandler         17
Angelina Jolie       16
Eddie Murphy         15
Samuel L. Jackson    15
                     ..
Leslie Bibb           1
Leonard Nimoy         1
Lennie James          1
Lena Olin             1
50 Cent               1
Length: 1505, dtype: int64

In [45]:
pd.Series(data[data.budget > data.budget.mean()].cast.str.split('|').sum()).value_counts()
# 1) filter data to find movies with big budget
# 2) split data in the cast column into the lists
# 3) Adding all lists in cast column into the one
# 4) Count each unique element in the list

Matt Damon            18
Adam Sandler          17
Angelina Jolie        16
Tom Cruise            15
Eddie Murphy          15
                      ..
Cole Konis             1
Edward James Olmos     1
Ed Speleers            1
Matthew Lillard        1
Peter Sohn             1
Length: 1505, dtype: int64

In [46]:
answers['16'] = '3. Matt Damon +'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [47]:
Nicolas_Cage_df = data[data.cast.str.contains('Nicolas Cage')] # filtered dataframe for films with Nikolas Cage
pd.Series(Nicolas_Cage_df['genres'].str.cat(sep='|').split('|')).value_counts()

Action             17
Thriller           15
Drama              12
Crime              10
Fantasy             8
Adventure           7
Comedy              6
Science Fiction     4
Family              3
Animation           3
Mystery             3
History             2
Romance             1
Horror              1
War                 1
dtype: int64

In [48]:
answers['17'] = '2. Action +'
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650) +',
 '2': '2. Gods and Generals (tt0279111) +',
 '3': '3. Winnie the Pooh (tt1449283) +',
 '4': '2. 110 +',
 '5': '1. 107 +',
 '6': '5. Avatar (tt0499549) +',
 '7': '5. The Lone Ranger (tt1210819) +',
 '8': '1. 1478 +',
 '9': '4. The Dark Knight (tt0468569) +',
 '10': '5. The Lone Ranger (tt1210819) +',
 '11': '3. Drama +',
 '12': '1. Drama +',
 '13': '5. Peter Jackson +',
 '14': '3. Robert Rodriguez +',
 '15': '3. Chris Hemsworth +',
 '16': '3. Matt Damon +',
 '17': '2. Action +'}

# 18. Самый убыточный фильм от Paramount Pictures

In [49]:
data[data.production_companies.str.contains('Paramount')].sort_values(by='profit').head(1)


,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,month,day,year,timefunc,title_length,num_words
925,tt0267626,100000000,35168966,K-19: The Widowmaker,Harrison Ford|Liam Neeson|Peter Sarsgaard|Joss...,Kathryn Bigelow,Fate has found its hero.,When Russia's first nuclear submarine malfunct...,138,Thriller|Drama|History,...,7/19/2002,6.0,2002,-64831034,7,19,2002,2002-07-19,20,23


In [50]:
answers['18'] = '1. K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [51]:
data.groupby(by='release_year').sum().sort_values(by='revenue',ascending=False).head(1)

,budget,revenue,runtime,vote_average,profit,title_length,num_words
release_year,,,,,,,
2015,6780630004,25449202382,13355,755.2,18668572378,1601,5524


In [52]:
data.pivot_table(index ='release_year', aggfunc='sum').sort_values(by='revenue').tail(1)

,budget,num_words,profit,revenue,runtime,title_length,vote_average
release_year,,,,,,,
2015,6780630004,5524,18668572378,25449202382,13355,1601,755.2


In [53]:
answers['19'] = '5. 2015 +'

# 20. Какой самый прибыльный год для студии Warner Bros?

In [54]:
WB_df=data[data.production_companies.str.contains('Warner')]
WB_df.pivot_table(index ='release_year', aggfunc='sum').sort_values(by='profit').tail(1)

,budget,num_words,profit,revenue,runtime,title_length,vote_average
release_year,,,,,,,
2014,947600000,672,2295464519,3243064519,1576,176,88.9


In [55]:
answers['20'] = '1. 2014 +'

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [56]:
data.pivot_table(values='original_title', index='month',aggfunc='count').sort_values(by='original_title')

,original_title
month,
1,110
2,135
5,140
7,142
11,146
6,147
4,149
3,156
8,161


In [57]:
##Solution usith method .month()
data.groupby(by=data['timefunc'].dt.month).count().sort_values(by = 'original_title')

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,month,day,year,timefunc,title_length,num_words
timefunc,,,,,,,,,,,,,,,,,,,,,
1,110,110,110,110,110,110,110,110,110,110,...,110,110,110,110,110,110,110,110,110,110
2,135,135,135,135,135,135,135,135,135,135,...,135,135,135,135,135,135,135,135,135,135
5,140,140,140,140,140,140,140,140,140,140,...,140,140,140,140,140,140,140,140,140,140
7,142,142,142,142,142,142,142,142,142,142,...,142,142,142,142,142,142,142,142,142,142
11,146,146,146,146,146,146,146,146,146,146,...,146,146,146,146,146,146,146,146,146,146
6,147,147,147,147,147,147,147,147,147,147,...,147,147,147,147,147,147,147,147,147,147
4,149,149,149,149,149,149,149,149,149,149,...,149,149,149,149,149,149,149,149,149,149
3,156,156,156,156,156,156,156,156,156,156,...,156,156,156,156,156,156,156,156,156,156
8,161,161,161,161,161,161,161,161,161,161,...,161,161,161,161,161,161,161,161,161,161


In [58]:
answers['21'] = '4. Сентябрь +'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [59]:
data.loc[data['timefunc'].dt.month.isin(['6','7','8'])].count()

imdb_id                 450
budget                  450
revenue                 450
original_title          450
cast                    450
director                450
tagline                 450
overview                450
runtime                 450
genres                  450
production_companies    450
release_date            450
vote_average            450
release_year            450
profit                  450
month                   450
day                     450
year                    450
timefunc                450
title_length            450
num_words               450
dtype: int64

In [60]:
answers['22'] = '2. 450 +'

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [61]:
data[data['timefunc'].dt.month.isin(['12','1','2'])].director.str.split('|').value_counts()

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1709, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[Peter Jackson]                       7
[Clint Eastwood]                      6
[Steven Soderbergh]                   6
[Shawn Levy]                          4
[Nancy Meyers]                        4
                                     ..
[Paul Tibbitt]                        1
[Florian Henckel von Donnersmarck]    1
[Niki Caro]                           1
[Mikael HÃ¥fstrÃ¶m]                   1
[Guy Ferland]                         1
Name: director, Length: 332, dtype: int64

In [62]:
answers['23'] = '5. Peter Jackson +'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [63]:
pd_df = data.copy()
pd_df['production_companies'] = pd_df.production_companies.str.split('|')
pd_df.explode('production_companies').groupby(['production_companies'])['title_length'].agg(['mean']).sort_values(by = 'mean')

,mean
production_companies,
Global Entertainment Group,2.0
Ixtlan Productions,2.0
XM2 Productions,2.0
Berlanti Productions,3.0
Everest Entertainment,3.0
...,...
Polsky Films,46.0
Museum Canada Productions,46.0
Dos Corazones,47.0


In [64]:
answers['24'] = '5. Four By Two Productions +'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [65]:
pd_df.explode('production_companies').groupby(['production_companies'])['num_words'].agg(['mean', 'max']).sort_values(by = 'mean')

,mean,max
production_companies,,
Motion Picture Corporation of America,11.0,11
Empire Pictures,11.0,11
Henceforth,13.0,13
Phantom Four,13.0,13
London Boulevard,13.0,13
...,...,...
Brookwell-McNamara Entertainment,156.0,156
98 MPH Productions,159.0,159
Heineken Branded Entertainment,161.0,161


In [66]:
answers['25'] = '3. Midnight Picture Show +'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [67]:
a = data.vote_average.quantile([0.99])#quantille method to get vote_average for 99% of the films

a[0.99]# 99% of the films has rating <= 7.8, or 1% of the best in rating > 7.8

7.8

In [68]:
data.original_title[(data.vote_average >= a[0.99])] 
#Series of films with rating >= 7.8 (slightly more than 1%, as there are several films rated 7.8)

9                                              Inside Out
28                                              Spotlight
34                                                   Room
118                                          Interstellar
119                               Guardians of the Galaxy
124                                            Big Hero 6
125                                    The Imitation Game
128                                             Gone Girl
138                              The Grand Budapest Hotel
155                              The Theory of Everything
177                                The Fault in Our Stars
283                                            Mr. Nobody
316                                              3 Idiots
370                                             Inception
496     The Lord of the Rings: The Fellowship of the Ring
599                                       The Dark Knight
863                 The Lord of the Rings: The Two Towers
872           

In [69]:
data.sort_values(by='vote_average', ascending = False).head(18)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,month,day,year,timefunc,title_length,num_words
599,tt0468569,185000000,1001921825,The Dark Knight,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,Drama|Action|Crime|Thriller,...,7/16/2008,8.1,2008,816921825,7,16,2008,2008-07-16,15,67
118,tt0816692,165000000,621752480,Interstellar,Matthew McConaughey|Jessica Chastain|Anne Hath...,Christopher Nolan,Mankind was born on Earth. It was never meant ...,Interstellar chronicles the adventures of a gr...,169,Adventure|Drama|Science Fiction,...,11/5/2014,8.0,2014,456752480,11,5,2014,2014-11-05,12,35
125,tt2084970,14000000,233555708,The Imitation Game,Benedict Cumberbatch|Keira Knightley|Matthew G...,Morten Tyldum,The true enigma was the man who cracked the code.,Based on the real life story of legendary cryp...,113,History|Drama|Thriller|War,...,11/14/2014,8.0,2014,219555708,11,14,2014,2014-11-14,18,46
9,tt2096673,175000000,853708609,Inside Out,Amy Poehler|Phyllis Smith|Richard Kind|Bill Ha...,Pete Docter,Meet the little voices inside your head.,"Growing up can be a bumpy road, and it's no ex...",94,Comedy|Animation|Family,...,6/9/2015,8.0,2015,678708609,6,9,2015,2015-06-09,10,112
34,tt3170832,6000000,35401758,Room,Brie Larson|Jacob Tremblay|Joan Allen|Sean Bri...,Lenny Abrahamson,Love knows no boundaries,Jack is a young boy of 5 years old who has liv...,117,Drama|Thriller,...,10/16/2015,8.0,2015,29401758,10,16,2015,2015-10-16,4,49
1183,tt0993846,100000000,392000694,The Wolf of Wall Street,Leonardo DiCaprio|Jonah Hill|Margot Robbie|Kyl...,Martin Scorsese,EARN. SPEND. PARTY.,A New York stockbroker refuses to cooperate in...,180,Crime|Drama|Comedy,...,12/25/2013,7.9,2013,292000694,12,25,2013,2013-12-25,23,29
128,tt2267998,61000000,369330363,Gone Girl,Ben Affleck|Rosamund Pike|Carrie Coon|Neil Pat...,David Fincher,You don't know what you've got 'til it's...,With his wife's disappearance having become th...,145,Mystery|Thriller|Drama,...,10/1/2014,7.9,2014,308330363,10,1,2014,2014-10-01,9,30
1191,tt2024544,20000000,187000000,12 Years a Slave,Chiwetel Ejiofor|Michael Fassbender|Lupita Nyo...,Steve McQueen,The extraordinary true story of Solomon Northup,"In the pre-Civil War United States, Solomon No...",134,Drama|History,...,10/18/2013,7.9,2013,167000000,10,18,2013,2013-10-18,16,61
119,tt2015381,170000000,773312399,Guardians of the Galaxy,Chris Pratt|Zoe Saldana|Dave Bautista|Vin Dies...,James Gunn,All heroes start somewhere.,"Light years from Earth, 26 years after being a...",121,Action|Science Fiction|Adventure,...,7/30/2014,7.9,2014,603312399,7,30,2014,2014-07-30,23,28
1081,tt0167260,94000000,1118888979,The Lord of the Rings: The Return of the King,Elijah Wood|Ian McKellen|Viggo Mortensen|Liv T...,Peter Jackson,The eye of the enemy is moving.,Aragorn is revealed as the heir to the ancient...,201,Adventure|Fantasy|Action,...,12/1/2003,7.9,2003,1024888979,12,1,2003,2003-12-01,45,45


In [70]:
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave +'

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [71]:
data['cast'] = data.cast.apply(func) # Used function to split cast column to list
data['cast_comb'] = data.cast.apply(func_comb) # New column in dataset with combination of the artist pairs
data.explode('cast_comb').value_counts('cast_comb') #Data DF exploded to use .value_counts().

cast_comb
(Daniel Radcliffe, Emma Watson)             8
(Daniel Radcliffe, Rupert Grint)            8
(Rupert Grint, Emma Watson)                 7
(Ben Stiller, Owen Wilson)                  6
(Johnny Depp, Helena Bonham Carter)         6
                                           ..
(Matthew McConaughey, Rene Russo)           1
(Matthew McConaughey, Reese Witherspoon)    1
(Matthew McConaughey, Powers Boothe)        1
(Matthew McConaughey, PenÃ©lope Cruz)       1
(50 Cent, Adewale Akinnuoye-Agbaje)         1
Length: 18121, dtype: int64

In [72]:
answers['27'] = '5. Daniel Radcliffe & Rupert Grint +'

ВАРИАНТ 2

In [73]:
list_of_pairs = []
import collections
c = collections.Counter() #Counter created

for elem in data['cast_comb'].tolist(): #cycle to go thru the list of pairs and add to the single list
    for pair in elem:
        list_of_pairs.append(pair)
for pair in list_of_pairs: #cycle to count the elements in the list
    c[pair] += 1
print(c.most_common()[0:3]) #printed first 3 of the most common pairs 

[(('Daniel Radcliffe', 'Rupert Grint'), 8), (('Daniel Radcliffe', 'Emma Watson'), 8), (('Rupert Grint', 'Emma Watson'), 7)]


# Submission

In [74]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650) +',
 '2': '2. Gods and Generals (tt0279111) +',
 '3': '3. Winnie the Pooh (tt1449283) +',
 '4': '2. 110 +',
 '5': '1. 107 +',
 '6': '5. Avatar (tt0499549) +',
 '7': '5. The Lone Ranger (tt1210819) +',
 '8': '1. 1478 +',
 '9': '4. The Dark Knight (tt0468569) +',
 '10': '5. The Lone Ranger (tt1210819) +',
 '11': '3. Drama +',
 '12': '1. Drama +',
 '13': '5. Peter Jackson +',
 '14': '3. Robert Rodriguez +',
 '15': '3. Chris Hemsworth +',
 '16': '3. Matt Damon +',
 '17': '2. Action +',
 '18': '1. K-19: The Widowmaker (tt0267626)',
 '19': '5. 2015 +',
 '20': '1. 2014 +',
 '21': '4. Сентябрь +',
 '22': '2. 450 +',
 '23': '5. Peter Jackson +',
 '24': '5. Four By Two Productions +',
 '25': '3. Midnight Picture Show +',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave +',
 '27': '5. Daniel Radcliffe & Rupert Grint +'}

In [75]:
# и убедиться что ни чего не пропустил)
len(answers)

27